How to convert a Google sheet to a static marketing website

templates google doc?

templates google sheet?

In [ ]:
var util = require('util');
var GoogleSpreadsheet = require('google-spreadsheet');

function getTemplates(doc) {
    if(typeof doc === 'string') {
        doc = new GoogleSpreadsheet(doc);
    }
    return util.promisify(doc.getInfo)()
        .then(info => {
            console.log('Loaded doc: ' + info.title + ' by ' + info.author.email);
            return info;
        })
        .then(s => s.worksheets.reduce((obj, w) => {
            var name = w.title.toLowerCase().replace(/\s*layout|\s*data|\s*template/ig, '');
            if( typeof obj[name] == 'undefined' ) obj[name] = {template: null, data: null};
            if( w.title.toLowerCase().includes('data') ) obj[name].data = w;
            else obj[name].template = w;
            return obj;
        }, {}))
}
module.exports = getTemplates;


test templates google sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get templates from sheet', () => {
    
    it('should have at least one page', () => {
        return getTemplates(docsId)
            .then(t => {
                console.log(JSON.stringify(Object.keys(t).reduce((obj, cur) => {
                    obj[cur] = {};
                    obj[cur].template = ((t[cur].template || {}).title || '').substr(0, 1000);
                    obj[cur].data = (t[cur].data ? ('yes: ' + t[cur].data.rowCount) : 'no');
                    return obj;
                }, {}), null, 4))
                assert(Object.keys(t).length > 0, 'should have a page called template');
            })
    })
    
    
})


get google data sheet?

google sheet array objects?


In [ ]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true});
var util = require('util');

function getDataSheet(worksheet) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            var columns = [];
            return cells.reduce((acc, o, arr) => {
                if (o.row == 1) {
                    columns[o.col] = o.value;
                    return acc;
                }
                if (!acc[o.row - 2]) {
                    acc[o.row - 2] = {};
                }
                acc[o.row - 2][columns[o.col]] = md.render(o.value);
                return acc;
            }, []);
        })
}
module.exports = getDataSheet;


test google data sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get data from data sheet', () => {
    
    it('should process at least one data sheet', () => {
        return getTemplates(docsId)
            .then(t => {
                var key = Object.keys(t).filter(k => t[k].data)[0];
                assert(typeof key != 'undefined', 'should have at least one data sheet');
                return getDataSheet(t[key].data);
            })
            .then(data => {
                console.log(data.length);
                console.log(data);
                assert(data.length > 0, 'should have data');
            })
    })
    
    
})


google sheet layout template?

Convert a google sheet page to an HTML template



In [1]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true});
var Mustache = require('mustache');

function getLayoutSheet(worksheet) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            // turn cells in to rows, TODO: can getCells options do this for me?
            var rows = cells.reduce((arr, c) => {
                if(typeof arr[c.row - 1] == 'undefined') arr[c.row - 1] = [];
                arr[c.row - 1][c.col - 1] = c.value;
                return arr;
            }, []);
        
            console.log(rows);
            
            // set object properties for mustache template
            var html = rows.reduce((arr, row, i) => {
                // use subsequent column for property values
                var props = {};
                var rowsHtml = row.map((c, j) => {
                    if(c.substr(0, 1) === ':') {
                        props[c.substr(1)] = row[j + 1];
                        return '';
                    } else if(j == 0 || j >= 1 && row[j - 1] && row[j - 1].substr(0, 1) !== ':') {
                        // render markdown content if it is not the value for the previous property
                        return `<div class="cell-${j}">${md.render(c)}</div>`;
                    } else {
                        return '';
                    }
                });
                
                // render mustache templates
                var rowTemplate = `
<div class="{{class}} col-{{colCount}}">
${rowsHtml.join('\n')}
</div>
`;
                Mustache.parse(rowTemplate);
                arr[i] = Mustache.render(rowTemplate, Object.assign(props, {
                    colCount: rowsHtml.length
                }));
                
                return arr;
            }, []);
        
            // render mustache page template
            return html.join('\n');
        })
}

module.exports = getLayoutSheet;

{ getLayoutSheet: [Function: getLayoutSheet],
  htmlSheetMarkup: [Function: htmlSheetMarkup] }

test google sheet template?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getLayoutSheet = importer.import('google sheet layout template');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get html from template', () => {
    
    it('should process at least one template', () => {
        return getTemplates(docsId)
            .then(t => {
                var key = Object.keys(t).filter(k => t[k].template)[0];
                assert(typeof key != 'undefined', 'should have at least one template');
                return getLayoutSheet(t[key].template);
            })
            .then(template => {
                console.log(template);
                assert(template.length > 0, 'should have a page called template');
            })
    })
    
    
})


output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');

function writeTemplate(className, html, data, sheetName, imports) {
    var formal = sheetName.split(/\s/ig)
        .map(t => t[0].toUpperCase() + t.substr(1))
        .join('')
        .replace(/\s*layout\s*/ig, '');
    var components = imports.map(i => i.split(/\s/ig)
        .map(t => t[0].toUpperCase() + t.substr(1))
        .join('')
        .replace(/\s*layout\s*/ig, ''));
    var importString = components.map(importFormal => 
        `import ${importFormal} from "./${importFormal}.vue";`
    ).join('\n');
    var name = sheetName.replace(/[^a-z]/ig, '');
    fs.writeFileSync(path.join(__dirname, '../src/views/generated/' + formal + '.vue'), `
<template>
    <div class="${formal.toLocaleLowerCase()} ${className}">
${html}
    </div>
</template>

<script>
${importString}

export default {
    name: '${name}',
    props: ['value'],
    components: { ${components.join(',')} },
    data() {
        return { 
            "${name}": JSON.parse(${JSON.stringify(JSON.stringify(data))})
        };
    },
};
</script>
`);
}

module.exports = writeTemplate;


test output sheet template?
